# Implementação do Algoritmo Árvore de Decisão
Este notebook contém a implementação do algoritmo Árvore de Decisão do zero, utilizando apenas a biblioteca NumPy, além de uma análise crítica do código gerado e dos resultados obtidos ao aplicá-lo no dataset Penguins.

## Prompts Utilizados e Respostas

### Prompt 1:

Implemente o algoritmo de árvore de decisão from scratch utilizando apenas a biblioteca NumPy para operações vetoriais. A implementação não pode utilizar outras bibliotecas prontas, o teste do algoritmo deve ser feito no dataset penguins_lter.csv na pasta raiz do projeto. Após a implementação e execução do algoritmo, printe na tela o desempenho do modelo utilizando as seguintes métricas: Matriz de confusão, Acurácia, Recall, Precisão, F1-score.

### Resposta 1:

Aqui está um código completo para a implementação de uma árvore de decisão usando apenas NumPy, incluindo funções para cálculo de entropia, ganho de informação, e divisão do dataset. Também incluiu um exemplo de como carregar o dataset pinguins_lter.csv, treinar o modelo, e calcular as métricas de desempenho.

In [275]:
import numpy as np
import pandas as pd

In [276]:
# Carregar o arquivo CSV
data = pd.read_csv('penguins_lter.csv')

# Limpar dados removendo entradas com valores ausentes e mapear a coluna Species para números
data_cleaned = data[['Culmen Length (mm)', 'Culmen Depth (mm)', 'Flipper Length (mm)', 'Body Mass (g)', 'Species']].dropna()

# Converter espécies para valores numéricos
species_mapping = {species: idx for idx, species in enumerate(data_cleaned['Species'].unique())}
data_cleaned['Species'] = data_cleaned['Species'].map(species_mapping)

# Separar features e alvo
X = data_cleaned[['Culmen Length (mm)', 'Culmen Depth (mm)', 'Flipper Length (mm)', 'Body Mass (g)']].values
y = data_cleaned['Species'].values

In [277]:

# Função para dividir os dados em treino e teste aleatoriamente
def train_test_split(X, y, test_size=0.2):
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)
    
    split_idx = int(X.shape[0] * (1 - test_size))
    
    X_train, X_test = X[indices[:split_idx]], X[indices[split_idx:]]
    y_train, y_test = y[indices[:split_idx]], y[indices[split_idx:]]
    
    return X_train, X_test, y_train, y_test

# Dividir os dados em treino e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [278]:

class DecisionTree:
    def __init__(self, max_depth=None, min_samples_split=2):  # Corrigir para __init__
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    def _build_tree(self, X, y, depth=0):
        num_samples, num_features = X.shape
        num_labels = len(np.unique(y))

        if (depth >= self.max_depth or num_labels == 1 or num_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return {"leaf": True, "value": leaf_value}

        best_feature, best_threshold = self._best_split(X, y)
        if best_feature is None:
            leaf_value = self._most_common_label(y)
            return {"leaf": True, "value": leaf_value}

        left_idxs, right_idxs = self._split(X[:, best_feature], best_threshold)
        left_subtree = self._build_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right_subtree = self._build_tree(X[right_idxs, :], y[right_idxs], depth + 1)

        return {
            "leaf": False,
            "feature": best_feature,
            "threshold": best_threshold,
            "left": left_subtree,
            "right": right_subtree,
        }

    def _best_split(self, X, y):
        num_samples, num_features = X.shape
        best_gini = float("inf")
        best_feature, best_threshold = None, None

        for feature in range(num_features):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                left_idxs, right_idxs = self._split(X[:, feature], threshold)

                if len(left_idxs) == 0 or len(right_idxs) == 0:
                    continue

                gini = self._gini_index(y, left_idxs, right_idxs)

                if gini < best_gini:
                    best_gini = gini
                    best_feature = feature
                    best_threshold = threshold

        return best_feature, best_threshold

    def _split(self, X_column, threshold):
        left_idxs = np.where(X_column <= threshold)[0]
        right_idxs = np.where(X_column > threshold)[0]
        return left_idxs, right_idxs

    def _gini_index(self, y, left_idxs, right_idxs):
        n = len(y)
        n_left, n_right = len(left_idxs), len(right_idxs)
        gini_left, gini_right = 0, 0

        if n_left > 0:
            gini_left = self._gini(y[left_idxs])

        if n_right > 0:
            gini_right = self._gini(y[right_idxs])

        return (n_left / n) * gini_left + (n_right / n) * gini_right

    def _gini(self, y):
        proportions = np.bincount(y) / len(y)
        return 1 - np.sum(proportions ** 2)

    def _most_common_label(self, y):
        return np.bincount(y).argmax()

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.tree) for x in X])

    def _traverse_tree(self, x, node):
        if node["leaf"]:
            return node["value"]

        if x[node["feature"]] <= node["threshold"]:
            return self._traverse_tree(x, node["left"])
        else:
            return self._traverse_tree(x, node["right"])


In [279]:

# Funções para calcular métricas de desempenho

def confusion_matrix(y_true, y_pred):
    num_classes = len(np.unique(y_true))
    matrix = np.zeros((num_classes, num_classes), dtype=int)
    for i in range(len(y_true)):
        matrix[y_true[i]][y_pred[i]] += 1
    return matrix

def accuracy(y_true, y_pred):
    return np.sum(y_true == y_pred) / len(y_true)

def precision(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    return np.diag(cm) / np.sum(cm, axis=0)

def recall(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    return np.diag(cm) / np.sum(cm, axis=1)

def f1_score(y_true, y_pred):
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return 2 * (p * r) / (p + r)

In [280]:
# Treinar o modelo e fazer previsões no conjunto de teste
tree = DecisionTree(max_depth=10)
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)


In [281]:
# Avaliar o desempenho
acc = accuracy(y_test, y_pred)
prec = np.nanmean(precision(y_test, y_pred))  # Evitar divisões por zero
rec = np.nanmean(recall(y_test, y_pred))
f1 = np.nanmean(f1_score(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)

print("Matriz de Confusão:")
print(cm)
print(f"Acurácia: {acc}")
print(f"Precisão: {prec}")
print(f"Recall: {rec}")
print(f"F1-score: {f1}")

Matriz de Confusão:
[[32  2  0]
 [ 1  8  0]
 [ 0  2 24]]
Acurácia: 0.927536231884058
Precisão: 0.8787878787878788
Recall: 0.9177140941846824
F1-score: 0.892376214167259



### Explicação do Código da Árvore de Decisão:

1. *Classe DecisionTree:*
   -  __init__: Configura o parâmetro de máxima profundidade (max_depth).
   -  fit: Treina a árvore de decisão com o dataset.
   -  _gini_index: Calcula o índice de Gini.
   -  predict: Faz previsões para dados de teste.

2. *Funções de Métricas:*
   -  accuracy: Calcula a acurácia.
   -  confusion_matrix: Gera uma matriz de confusão.
   -  precision: Calcula a precisão da previsão.
   -  recall: Calcula o recall da previsão.
   -  f1_score: Calcula o F1-score.

3. *Carregamento do Dataset:*
   -  Leitura dos dados: Carrega o dataset de penguins_lter.csv, seleciona colunas importantes e tira valores nulos.

4. *Execução do Algoritmo*:
   - O dataset é embaralhado aleatoriamente e dividido em 80% para treino e 20% para teste.
   - O modelo árvore de decisão é treinado e testado.
   - As métricas são calculadas e exibidas.

### Exemplo de saída
Matriz de Confusão:<br>
[[32  2  0]<br>
 [ 1  8  0]<br>
 [ 0  2 24]]<br>
Acurácia: 0.927536231884058<br>
Precisão: 0.8787878787878788<br>
Recall: 0.9177140941846824<br>
F1-score: 0.892376214167259

## Análise Crítica do Código Gerado

Primeiramente, testamos o algoritmo com o dataset Iris, porém tivemos um resultado perfeito, irreal. Portanto, optamos, novamente, por usarmos o dataset Pinguins. Trazendo métricas mais realistas, porém ainda altas. Dessa forma, testamos o cross-validation para a tentativa de métricas mais realistas, entretanto não mudou muito o resultado final. Então, acreditamos que com a poda da árvore, poderíamos obter essas métricas desejadas.

### Pontos Fortes
- *Código simples e fácil de visualizar:* O código é direto e fácil de entender, com métricas de profundidade máxima.

### Potenciais Problemas/Diferenças
- *Eficiência Computacional:* A implementação pode ser menos eficiente em termos de tempo de execução.
- *Generalização:* O algoritmo atual não implementa a poda da árvore, o que pode levar a overfitting em datasets maiores.

### Opções de Projeto e Impacto
- *Implementar Poda:* Incluir um mecanismo para poda da árvore poderia ajudar a melhorar a generalização do modelo.

### Sugestões de Melhorias
- *Implementar Poda:* Como dito anteriormente, podemos implementar a poda para obter resultados e métricas mais realistas.